In [ ]:
import numpy as np
import pandas as pd
import random

import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

import matplotlib.pyplot as plt

# фіксуємо random seed, щоб все було відтворювано
SEED = 42
np.random.seed(SEED)
random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)



Using device: cpu


In [ ]:
# Block 2: paths & hyperparameters

base_dir = r"C:\Users\Anton\OneDrive\Desktop\КПІлабки\lab3 Габунія А.П. ФІ-51мн"

train_dir = os.path.join(base_dir, "seg_train", "seg_train")
test_dir  = os.path.join(base_dir, "seg_test")   # якщо немає test -> можна взяти частину train

image_size = 64         # зменшуємо, щоб було швидко
batch_size = 64
num_epochs = 5          # якщо повільно - зменшити до 3
learning_rate = 1e-3

max_images_per_class = 500   # обмеження, щоб тренувалось недовго


In [50]:
# Block 3: datasets & strong simplification

transform = transforms.Compose([
    transforms.Resize((image_size, image_size)),
    transforms.ToTensor(),
])

full_train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
full_test_dataset  = datasets.ImageFolder(root=test_dir,  transform=transform)

print("Full train size:", len(full_train_dataset))
print("Full test size:", len(full_test_dataset))
print("Classes:", full_train_dataset.classes)

def limit_per_class(dataset, max_per_class):
    indices = []
    counter = defaultdict(int)
    for idx, (_, label) in enumerate(dataset):
        if counter[label] < max_per_class:
            indices.append(idx)
            counter[label] += 1
    return Subset(dataset, indices)

train_dataset = limit_per_class(full_train_dataset, max_images_per_class)
test_dataset  = limit_per_class(full_test_dataset,  max_images_per_class)

print("Reduced train size:", len(train_dataset))
print("Reduced test size:", len(test_dataset))

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False, num_workers=0)


Full train size: 14034
Full test size: 3000
Classes: ['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']
Reduced train size: 6000
Reduced test size: 1000


In [51]:
# Block 4: simple CNN model

class SimpleCNN(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 16, kernel_size=3, padding=1),  # 3 -> 16
            nn.ReLU(),
            nn.MaxPool2d(2),                             # 64 -> 32

            nn.Conv2d(16, 32, kernel_size=3, padding=1), # 16 -> 32
            nn.ReLU(),
            nn.MaxPool2d(2),                             # 32 -> 16

            nn.Conv2d(32, 64, kernel_size=3, padding=1), # 32 -> 64
            nn.ReLU(),
            nn.MaxPool2d(2),                             # 16 -> 8
        )
        # після трьох пулінгів: 64 каналів, розмір фічмапи 8x8 => 64*8*8
        self.classifier = nn.Sequential(
            nn.Flatten(),
            nn.Linear(64 * 8 * 8, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = self.classifier(x)
        return x

num_classes = len(full_train_dataset.classes)
model = SimpleCNN(num_classes).to(device)
print(model)


SimpleCNN(
  (features): Sequential(
    (0): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(16, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU()
    (8): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=4096, out_features=128, bias=True)
    (2): ReLU()
    (3): Linear(in_features=128, out_features=6, bias=True)
  )
)


In [52]:
# Block 5: loss & optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)


In [53]:
# Block 6: evaluation function on test set

def evaluate(model, data_loader):
    model.eval()
    correct = 0
    total = 0
    running_loss = 0.0

    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item() * images.size(0)

            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    avg_loss = running_loss / total
    accuracy = correct / total
    return avg_loss, accuracy


In [54]:
# Block 7: training loop

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        images = images.to(device)
        labels = labels.to(device)

        optimizer.zero_grad()

        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)

        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    train_loss = running_loss / total
    train_acc = correct / total

    test_loss, test_acc = evaluate(model, test_loader)

    print(f"Epoch [{epoch+1}/{num_epochs}] "
          f"Train loss: {train_loss:.4f}, acc: {train_acc:.4f} | "
          f"Test loss: {test_loss:.4f}, acc: {test_acc:.4f}")


Epoch [1/10] Train loss: 1.2558, acc: 0.5052 | Test loss: 2.8175, acc: 0.2070
Epoch [2/10] Train loss: 0.9696, acc: 0.6257 | Test loss: 3.6209, acc: 0.2540
Epoch [3/10] Train loss: 0.8510, acc: 0.6800 | Test loss: 3.0534, acc: 0.3530
Epoch [4/10] Train loss: 0.8011, acc: 0.6992 | Test loss: 2.8525, acc: 0.2460
Epoch [5/10] Train loss: 0.7490, acc: 0.7243 | Test loss: 3.3297, acc: 0.2770
Epoch [6/10] Train loss: 0.6796, acc: 0.7495 | Test loss: 3.1111, acc: 0.2780
Epoch [7/10] Train loss: 0.6505, acc: 0.7597 | Test loss: 3.3953, acc: 0.2400
Epoch [8/10] Train loss: 0.5868, acc: 0.7837 | Test loss: 3.9756, acc: 0.3530
Epoch [9/10] Train loss: 0.5606, acc: 0.7892 | Test loss: 3.8069, acc: 0.2960
Epoch [10/10] Train loss: 0.5246, acc: 0.8077 | Test loss: 3.7437, acc: 0.2930
